# Part X: Advanced Topics and Ecosystem

## Chapter 24: Advanced Patterns

Production FastAPI applications often require integration with legacy systems, custom documentation needs, or specific real-time communication patterns. This chapter covers advanced architectural patterns including Server-Sent Events for efficient unidirectional streaming, custom OpenAPI schema modifications for specialized documentation requirements, and mounting sub-applications to incrementally migrate from other frameworks or integrate specialized tools.

---

### 24.1 Server-Sent Events (SSE): Streaming Updates to Clients

Server-Sent Events (SSE) provide a standardized way for servers to push real-time updates to clients over HTTP. Unlike WebSockets, SSE is unidirectional (server-to-client only), uses standard HTTP, and includes automatic reconnection with event IDs.

#### SSE vs WebSockets Comparison

```
┌─────────────────────────────────────────────────────────────────┐
│              SSE vs WebSockets: When to Use Which                │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Server-Sent Events (SSE)                                        │
│  ─────────────────────────────────────────────────────────────  │
│  Protocol: HTTP/1.1 or HTTP/2 (standard)                        │
│  Direction: Server → Client only (unidirectional)                │
│  Transport: Text-only (UTF-8)                                   │
│  Reconnection: Built-in with EventSource API                     │
│  Browser Support: Native EventSource (no library needed)         │
│                                                                  │
│  Best For:                                                       │
│   • Live notifications, alerts                                   │
│   • Real-time dashboards, metrics streaming                      │
│   • Log streaming, progress updates                              │
│   • Stock tickers, price updates                                 │
│   • Social media feeds (new posts)                               │
│                                                                  │
│  Limitations:                                                    │
│   • 6 connections per browser (HTTP/1.1 limit)                   │
│   • No binary data (base64 encode if needed)                     │
│   • Client cannot send messages over same connection            │
│                                                                  │
│  ─────────────────────────────────────────────────────────────  │
│                                                                  │
│  WebSockets                                                      │
│  ─────────────────────────────────────────────────────────────  │
│  Protocol: ws:// / wss:// (separate from HTTP)                   │
│  Direction: Bidirectional (client ↔ server)                      │
│  Transport: Binary and text                                      │
│  Reconnection: Must implement manually                           │
│  Browser Support: Native WebSocket API                           │
│                                                                  │
│  Best For:                                                       │
│   • Chat applications (bidirectional)                            │
│   • Collaborative editing (locks, cursors)                       │
│   • Real-time games (input + state updates)                        │
│   • Video/audio streaming control                                  │
│   • Any scenario requiring client→server messages                │
│                                                                  │
│  Decision: Use SSE unless you need bidirectional communication   │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Implementing SSE in FastAPI

```python
# sse_implementation.py - Server-Sent Events
from fastapi import FastAPI, Request
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware
import asyncio
import json
from datetime import datetime
from typing import AsyncGenerator
import logging

logger = logging.getLogger(__name__)
app = FastAPI()

# CORS required for SSE (browser security)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Restrict in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ═════════════════════════════════════════════════════════════════
# Basic SSE Endpoint
# ═════════════════════════════════════════════════════════════════

@app.get("/sse/events")
async def events_stream(request: Request):
    """
    Basic SSE endpoint streaming events.
    
    Client connects and receives events until connection closes.
    """
    async def event_generator() -> AsyncGenerator[str, None]:
        """Generate SSE formatted events."""
        event_id = 0
        
        while True:
            # Check if client disconnected
            if await request.is_disconnected():
                logger.info("Client disconnected from SSE")
                break
            
            event_id += 1
            
            # SSE format: field: value\n
            # Required fields: data (can be multiline)
            # Optional: id, event, retry
            
            data = {
                "timestamp": datetime.utcnow().isoformat(),
                "message": f"Event #{event_id}",
                "status": "active"
            }
            
            # SSE format
            yield f"id: {event_id}\n"
            yield f"event: update\n"  # Event type
            yield f"data: {json.dumps(data)}\n\n"  # Double newline ends event
            
            # Wait before next event
            await asyncio.sleep(2)
    
    return StreamingResponse(
        event_generator(),
        media_type="text/event-stream",
        headers={
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
            "X-Accel-Buffering": "no",  # Disable nginx buffering
        }
    )


# ═════════════════════════════════════════════════════════════════
# SSE with Specific Event Types
# ═════════════════════════════════════════════════════════════════

class SSEManager:
    """
    Manager for SSE connections with typed events.
    
    Supports multiple event types and connection management.
    """
    
    def __init__(self):
        self.connections: list[StreamingResponse] = []
    
    async def send_event(
        self,
        event_type: str,
        data: dict,
        event_id: str = None
    ):
        """Send event to all connected clients."""
        # In real implementation, track active generators
        pass
    
    def format_event(
        self,
        event_type: str,
        data: dict,
        event_id: str = None,
        retry: int = None
    ) -> str:
        """
        Format data as SSE event string.
        
        Args:
            event_type: Event name (client listens for this)
            data: JSON-serializable data
            event_id: ID for replay/resume
            retry: Reconnection delay in milliseconds
        """
        lines = []
        
        if event_id:
            lines.append(f"id: {event_id}")
        
        if event_type:
            lines.append(f"event: {event_type}")
        
        if retry is not None:
            lines.append(f"retry: {retry}")
        
        # Data can be multiline (each line prefixed with data:)
        data_str = json.dumps(data)
        for line in data_str.split('\n'):
            lines.append(f"data: {line}")
        
        lines.append("")  # Empty line terminates event
        lines.append("")  # Double newline
        
        return "\n".join(lines)


sse_manager = SSEManager()


@app.get("/sse/notifications/{user_id}")
async def notifications_stream(user_id: str, request: Request):
    """
    User-specific notification stream.
    
    Client JavaScript:
    const source = new EventSource('/sse/notifications/123');
    
    source.addEventListener('notification', (e) => {
        const data = JSON.parse(e.data);
        console.log('Notification:', data);
    });
    
    source.addEventListener('system', (e) => {
        console.log('System message:', e.data);
    });
    
    // Automatic reconnection with Last-Event-ID header
    """
    async def notification_generator() -> AsyncGenerator[str, None]:
        last_event_id = request.headers.get("last-event-id", "0")
        event_id = int(last_event_id)
        
        # Send initial connection event
        yield sse_manager.format_event(
            "system",
            {"message": "Connected", "user_id": user_id},
            event_id=str(event_id)
        )
        
        # Simulate checking for notifications
        while True:
            if await request.is_disconnected():
                break
            
            # Check for new notifications (from Redis, DB, etc.)
            notification = await get_notification_for_user(user_id)
            
            if notification:
                event_id += 1
                yield sse_manager.format_event(
                    "notification",
                    notification,
                    event_id=str(event_id)
                )
            
            # Heartbeat to keep connection alive
            yield ": heartbeat\n\n"  # Comment line (ignored by client)
            
            await asyncio.sleep(1)
    
    return StreamingResponse(
        notification_generator(),
        media_type="text/event-stream",
        headers={
            "Cache-Control": "no-cache",
            "Connection": "keep-alive",
        }
    )


# ═════════════════════════════════════════════════════════════════
# SSE with Database Integration (Real Example)
# ═════════════════════════════════════════════════════════════════

@app.get("/sse/job-progress/{job_id}")
async def job_progress_stream(job_id: str, request: Request):
    """
    Stream job progress updates.
    
    Client receives updates as job status changes in database.
    Supports resuming from last known event ID.
    """
    async def progress_generator() -> AsyncGenerator[str, None]:
        last_id = int(request.headers.get("last-event-id", "0"))
        
        while True:
            if await request.is_disconnected():
                logger.info(f"Client disconnected from job {job_id}")
                break
            
            # Query current status
            job = await get_job_status(job_id)
            
            if not job:
                yield sse_manager.format_event(
                    "error",
                    {"message": "Job not found"},
                    event_id=str(last_id + 1)
                )
                break
            
            # Only send if status changed or new event
            current_event_id = job["updated_at"].timestamp()
            
            if current_event_id > last_id:
                last_id = current_event_id
                
                yield sse_manager.format_event(
                    "progress",
                    {
                        "job_id": job_id,
                        "status": job["status"],
                        "progress": job["progress_percent"],
                        "message": job["status_message"],
                        "completed": job["status"] == "completed",
                        "result": job.get("result")
                    },
                    event_id=str(int(current_event_id))
                )
                
                # End stream if job complete
                if job["status"] in ["completed", "failed", "cancelled"]:
                    break
            
            await asyncio.sleep(1)
    
    return StreamingResponse(
        progress_generator(),
        media_type="text/event-stream"
    )


# ═════════════════════════════════════════════════════════════════
# Broadcasting SSE with Redis (Multi-Server)
# ═════════════════════════════════════════════════════════════════

from redis.asyncio import Redis

redis_client = Redis.from_url("redis://localhost:6379/0")

@app.get("/sse/broadcast")
async def broadcast_stream(request: Request):
    """
    Subscribe to broadcast channel via Redis.
    
    Scales across multiple FastAPI instances.
    """
    async def broadcast_generator() -> AsyncGenerator[str, None]:
        # Subscribe to Redis channel
        pubsub = redis_client.pubsub()
        await pubsub.subscribe("broadcast:channel")
        
        try:
            async for message in pubsub.listen():
                if message["type"] == "message":
                    data = json.loads(message["data"])
                    
                    yield sse_manager.format_event(
                        data["event_type"],
                        data["payload"]
                    )
        finally:
            await pubsub.unsubscribe()
    
    return StreamingResponse(
        broadcast_generator(),
        media_type="text/event-stream"
    )


# Trigger broadcast from elsewhere
@app.post("/broadcast")
async def trigger_broadcast(message: dict):
    """Publish message to all connected SSE clients."""
    await redis_client.publish(
        "broadcast:channel",
        json.dumps({
            "event_type": "broadcast",
            "payload": message
        })
    )
    return {"status": "broadcasted"}
```

**SSE Client Implementation:**

```javascript
// sse-client.js - Browser EventSource API

// Basic connection
const source = new EventSource('/sse/events');

// Handle specific event types
source.addEventListener('update', (event) => {
    const data = JSON.parse(event.data);
    console.log('Update:', data);
    console.log('Event ID:', event.lastEventId); // For resume
});

source.addEventListener('system', (event) => {
    console.log('System:', event.data);
});

// Error handling (auto-reconnects by default)
source.onerror = (error) => {
    console.error('SSE error:', error);
    // EventSource will auto-reconnect with Last-Event-ID header
};

// Manual close
source.close();

// With custom headers (requires fetch polyfill for auth tokens)
const connectSSE = (url, token) => {
    const eventSource = new EventSource(url, {
        headers: { 'Authorization': `Bearer ${token}` } // Note: Limited browser support
    });
    return eventSource;
};
```

---

### 24.2 Custom OpenAPI: Modifying the Schema Manually

FastAPI automatically generates OpenAPI schemas from your code. However, production APIs often need custom modifications: deprecated endpoints, vendor extensions, examples, or manual schema adjustments for backward compatibility.

#### Customizing OpenAPI Schema

```python
# custom_openapi.py - Schema customization
from fastapi import FastAPI, Path, Query
from fastapi.openapi.utils import get_openapi
from fastapi.responses import JSONResponse
import json

app = FastAPI(
    title="Custom API",
    version="1.0.0",
    description="API with custom OpenAPI modifications"
)

# ═════════════════════════════════════════════════════════════════
# Method 1: Route-Level OpenAPI Customization
# ═════════════════════════════════════════════════════════════════

@app.get(
    "/items/{item_id}",
    summary="Get item",
    description="Retrieve a single item by ID",
    response_description="The requested item",
    deprecated=False,  # Mark as deprecated to warn users
    openapi_extra={
        "x-code-samples": [  # Custom vendor extension
            {
                "lang": "Python",
                "source": "import requests\nresponse = requests.get('https://api.example.com/items/123')"
            },
            {
                "lang": "curl",
                "source": "curl -X GET 'https://api.example.com/items/123' -H 'Authorization: Bearer TOKEN'"
            }
        ],
        "x-internal": True,  # Custom metadata
    }
)
async def get_item(
    item_id: str = Path(
        ...,
        description="The item ID",
        example="123e4567-e89b-12d3-a456-426614174000"
    )
):
    return {"item_id": item_id}


# ═════════════════════════════════════════════════════════════════
# Method 2: Custom OpenAPI Schema Generation
# ═════════════════════════════════════════════════════════════════

def custom_openapi():
    """
    Completely customize the OpenAPI schema.
    
    Called once at startup, cached for subsequent requests.
    """
    if app.openapi_schema:
        return app.openapi_schema
    
    # Generate base schema
    openapi_schema = get_openapi(
        title="Custom API",
        version="1.0.0",
        description="Production API with custom documentation",
        routes=app.routes,
    )
    
    # ═════════════════════════════════════════════════════════════
    # Custom Modifications
    # ═════════════════════════════════════════════════════════════
    
    # 1. Add global servers
    openapi_schema["servers"] = [
        {"url": "https://api.production.com", "description": "Production"},
        {"url": "https://api.staging.com", "description": "Staging"},
        {"url": "http://localhost:8000", "description": "Local Development"}
    ]
    
    # 2. Add security schemes
    openapi_schema["components"]["securitySchemes"] = {
        "bearerAuth": {
            "type": "http",
            "scheme": "bearer",
            "bearerFormat": "JWT",
            "description": "Enter JWT token"
        },
        "apiKeyAuth": {
            "type": "apiKey",
            "in": "header",
            "name": "X-API-Key"
        }
    }
    
    # 3. Add security requirement globally
    openapi_schema["security"] = [
        {"bearerAuth": []}
    ]
    
    # 4. Add custom headers to all responses
    for path_data in openapi_schema["paths"].values():
        for operation in path_data.values():
            if "responses" in operation:
                for response in operation["responses"].values():
                    if "headers" not in response:
                        response["headers"] = {}
                    response["headers"]["X-Request-ID"] = {
                        "description": "Unique request identifier",
                        "schema": {"type": "string"}
                    }
    
    # 5. Mark specific routes as deprecated
    if "/old-endpoint" in openapi_schema["paths"]:
        openapi_schema["paths"]["/old-endpoint"]["get"]["deprecated"] = True
        openapi_schema["paths"]["/old-endpoint"]["get"]["description"] = \
            "⚠️ **Deprecated**: Use `/new-endpoint` instead."
    
    # 6. Add custom tags with descriptions
    openapi_schema["tags"] = [
        {
            "name": "Users",
            "description": "User management operations",
            "externalDocs": {
                "description": "Learn more",
                "url": "https://docs.example.com/users"
            }
        },
        {
            "name": "Legacy",
            "description": "Deprecated endpoints maintained for backward compatibility"
        }
    ]
    
    # 7. Add custom schema examples
    if "schemas" in openapi_schema["components"]:
        if "User" in openapi_schema["components"]["schemas"]:
            openapi_schema["components"]["schemas"]["User"]["example"] = {
                "id": "123e4567-e89b-12d3-a456-426614174000",
                "username": "alice",
                "email": "alice@example.com",
                "created_at": "2024-01-15T10:30:00Z"
            }
    
    # 8. Add webhooks (OpenAPI 3.1.0)
    openapi_schema["webhooks"] = {
        "userCreated": {
            "post": {
                "summary": "User created webhook",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {"$ref": "#/components/schemas/User"}
                        }
                    }
                }
            }
        }
    }
    
    # 9. Add external documentation
    openapi_schema["externalDocs"] = {
        "description": "API Documentation",
        "url": "https://docs.example.com/api"
    }
    
    # Cache the schema
    app.openapi_schema = openapi_schema
    return app.openapi_schema


# Replace default openapi generation
app.openapi = custom_openapi


# ═════════════════════════════════════════════════════════════════
# Method 3: Custom Response Schema with Examples
# ═════════════════════════════════════════════════════════════════

from pydantic import BaseModel, Field

class ErrorResponse(BaseModel):
    """Standardized error response."""
    error: str = Field(..., example="validation_error")
    message: str = Field(..., example="Invalid input data")
    details: dict = Field(
        default_factory=dict,
        example={"field": "email", "issue": "invalid format"}
    )
    
    class Config:
        schema_extra = {
            "examples": {
                "validation_error": {
                    "summary": "Validation Error",
                    "value": {
                        "error": "validation_error",
                        "message": "Request validation failed",
                        "details": {"username": "already taken"}
                    }
                },
                "not_found": {
                    "summary": "Not Found",
                    "value": {
                        "error": "not_found",
                        "message": "Resource not found",
                        "details": {"id": "123"}
                    }
                }
            }
        }


@app.post(
    "/items/",
    responses={
        201: {
            "description": "Item created successfully",
            "content": {
                "application/json": {
                    "example": {
                        "id": "123",
                        "name": "New Item",
                        "created": True
                    }
                }
            }
        },
        422: {
            "description": "Validation Error",
            "model": ErrorResponse
        },
        500: {
            "description": "Internal Server Error",
            "content": {
                "application/json": {
                    "example": {
                        "error": "internal_error",
                        "message": "An unexpected error occurred"
                    }
                }
            }
        }
    }
)
async def create_item(item: ItemCreate):
    """Create with custom response documentation."""
    return {"id": "123", "name": item.name, "created": True}


# ═════════════════════════════════════════════════════════════════
# Method 4: Custom OpenAPI Route (Advanced)
# ═════════════════════════════════════════════════════════════════

from fastapi.openapi.docs import get_swagger_ui_html
from fastapi.staticfiles import StaticFiles

# Mount custom documentation assets
app.mount("/static", StaticFiles(directory="static"), name="static")

@app.get("/docs", include_in_schema=False)
async def custom_swagger_ui_html():
    """
    Custom Swagger UI with branding.
    
    Override default /docs endpoint.
    """
    return get_swagger_ui_html(
        openapi_url="/openapi.json",
        title="Custom API Documentation",
        swagger_js_url="/static/swagger-ui-bundle.js",
        swagger_css_url="/static/swagger-ui.css",
        swagger_favicon_url="/static/favicon.ico",
        init_oauth={
            "clientId": "your-client-id",
            "appName": "Your App"
        }
    )


# Custom OpenAPI JSON endpoint with caching
@app.get("/openapi.json", include_in_schema=False)
async def get_openapi_json():
    """Serve custom OpenAPI schema."""
    return JSONResponse(
        content=app.openapi(),
        headers={"Cache-Control": "max-age=3600"}  # Cache for 1 hour
    )
```

---

### 24.3 Sub-Applications: Mounting Other ASGI Apps

FastAPI supports mounting other ASGI applications, enabling gradual migrations from Django, Flask, or integration of specialized tools like Django Admin, Prometheus metrics, or static file servers.

#### Mounting Sub-Applications

```python
# sub_applications.py - Mounting other ASGI apps
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from fastapi.middleware.wsgi import WSGIMiddleware
from fastapi.middleware.httpsredirect import HTTPSRedirectMiddleware
import uvicorn

# Main FastAPI application
app = FastAPI(title="Main Application")

# ═════════════════════════════════════════════════════════════════
# Mounting Static Files
# ═════════════════════════════════════════════════════════════════

# Serve static files at /static
app.mount("/static", StaticFiles(directory="static"), name="static")

# Serve uploaded files with specific configuration
app.mount(
    "/uploads",
    StaticFiles(
        directory="uploads",
        html=False,  # Don't serve index.html
        check_dir=True  # Verify directory exists
    ),
    name="uploads"
)


# ═════════════════════════════════════════════════════════════════
# Mounting Another FastAPI App (Modular Architecture)
# ═════════════════════════════════════════════════════════════════

# Create sub-application
admin_app = FastAPI(
    title="Admin API",
    description="Administrative endpoints",
    version="1.0.0",
    docs_url="/docs",
    openapi_url="/openapi.json"
)

@admin_app.get("/users")
async def admin_list_users():
    """Admin-only endpoint."""
    return {"users": []}

@admin_app.get("/stats")
async def admin_stats():
    """System statistics."""
    return {"stats": {}}

# Mount with prefix
app.mount("/admin", admin_app)

# Now accessible at:
# /admin/docs (Swagger UI)
# /admin/users
# /admin/stats


# ═════════════════════════════════════════════════════════════════
# Mounting Flask Application (Legacy Integration)
# ═════════════════════════════════════════════════════════════════

from flask import Flask, jsonify

# Existing Flask app
flask_app = Flask(__name__)

@flask_app.route("/legacy/users")
def flask_users():
    """Legacy Flask endpoint."""
    return jsonify({"users": "from flask"})

@flask_app.route("/legacy/data")
def flask_data():
    """Another legacy endpoint."""
    return jsonify({"data": "legacy data"})

# Wrap Flask app with WSGIMiddleware
app.mount("/legacy", WSGIMiddleware(flask_app))

# Flask routes now accessible at:
# /legacy/users
# /legacy/data


# ═════════════════════════════════════════════════════════════════
# Mounting Django Application
# ═════════════════════════════════════════════════════════════════

import os
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'myproject.settings')

import django
django.setup()

from django.core.handlers.asgi import ASGIHandler

# Django ASGI application
django_app = ASGIHandler()

# Mount Django admin and API
app.mount("/django", django_app)

# Django URLs accessible at /django/


# ═════════════════════════════════════════════════════════════════
# Mounting Prometheus Metrics
# ═════════════════════════════════════════════════════════════════

from prometheus_client import make_asgi_app

# Create Prometheus metrics app
metrics_app = make_asgi_app()

# Mount at /metrics
app.mount("/metrics", metrics_app)

# Now /metrics returns Prometheus format metrics


# ═════════════════════════════════════════════════════════════════
# Custom ASGI Sub-Application
# ═════════════════════════════════════════════════════════════════

from starlette.types import Receive, Scope, Send

class CustomASGIApp:
    """
    Custom ASGI application for specialized handling.
    
    Useful for WebSocket proxies, custom protocols, etc.
    """
    
    async def __call__(self, scope: Scope, receive: Receive, send: Send):
        if scope["type"] == "http":
            await self.handle_http(scope, receive, send)
        elif scope["type"] == "websocket":
            await self.handle_websocket(scope, receive, send)
    
    async def handle_http(self, scope, receive, send):
        """Handle HTTP requests."""
        await send({
            "type": "http.response.start",
            "status": 200,
            "headers": [[b"content-type", b"application/json"]]
        })
        await send({
            "type": "http.response.body",
            "body": b'{"custom": "response"}'
        })
    
    async def handle_websocket(self, scope, receive, send):
        """Handle WebSocket connections."""
        await send({"type": "websocket.accept"})
        await send({
            "type": "websocket.send",
            "text": '{"message": "connected"}'
        })

# Mount custom app
app.mount("/custom", CustomASGIApp())


# ═════════════════════════════════════════════════════════════════
# Conditional Routing with Host-Based Mounting
# ═════════════════════════════════════════════════════════════════

from starlette.routing import Host

# Different apps for different subdomains
api_app = FastAPI()
docs_app = FastAPI()

@api_app.get("/")
async def api_root():
    return {"api": "version 1"}

@docs_app.get("/")
async def docs_root():
    return {"docs": "documentation site"}

# Host-based routing (requires reverse proxy setup)
# api.example.com -> api_app
# docs.example.com -> docs_app

# In practice, use nginx for host routing:
# server {
#     server_name api.example.com;
#     location / { proxy_pass http://localhost:8000; }
# }


# ═════════════════════════════════════════════════════════════════
# Middleware Specific to Sub-Application
# ═════════════════════════════════════════════════════════════════

# Create sub-app with its own middleware
internal_app = FastAPI()

@internal_app.middleware("http")
async def internal_auth(request, call_next):
    """Middleware only for internal routes."""
    # Check internal API key
    api_key = request.headers.get("X-Internal-Key")
    if api_key != "secret-internal-key":
        from fastapi import HTTPException
        raise HTTPException(status_code=403, detail="Forbidden")
    
    response = await call_next(request)
    return response

@internal_app.get("/health")
async def internal_health():
    """Internal health check."""
    return {"status": "ok", "internal": True}

app.mount("/internal", internal_app)


# ═════════════════════════════════════════════════════════════════
# Graceful Degradation for Mounted Apps
# ═════════════════════════════════════════════════════════════════

@app.exception_handler(Exception)
async def mounted_app_exception_handler(request, exc):
    """
    Handle errors from mounted sub-applications.
    
    Prevents sub-app crashes from bringing down main app.
    """
    logger.error(f"Error in mounted app: {exc}")
    
    return JSONResponse(
        status_code=500,
        content={
            "error": "Service temporarily unavailable",
            "path": request.url.path
        }
    )
```

**Sub-Application Best Practices:**

1. **Path Prefixes**: Always use descriptive prefixes (`/admin`, `/api/v1`, `/legacy`) to avoid route conflicts
2. **Independent Documentation**: Sub-apps can have their own `/docs` endpoints
3. **Middleware Isolation**: Middleware on sub-apps only affects that sub-app
4. **Error Boundaries**: Implement exception handlers at mount point to isolate failures
5. **Static Files**: Mount static files at specific paths, not root (`/`)

---

### Summary

In this chapter, you mastered advanced FastAPI patterns:

1. **Server-Sent Events (SSE)**: Implemented unidirectional server-to-client streaming using `StreamingResponse` with `text/event-stream` media type, supporting automatic reconnection with event IDs, heartbeat keepalives, and Redis-backed broadcasting for multi-server scalability.

2. **Custom OpenAPI**: Modified auto-generated schemas using `app.openapi` override, added custom vendor extensions (`x-code-samples`), configured global security schemes, marked deprecated endpoints, injected custom headers in responses, and provided rich request/response examples.

3. **Sub-Applications**: Mounted other ASGI applications including FastAPI modules for modular architecture, Flask apps via `WSGIMiddleware` for legacy integration, Django via `ASGIHandler`, Prometheus metrics endpoints, and custom ASGI handlers for specialized protocols.

**Advanced Patterns Checklist:**
- Use SSE for unidirectional streaming (simpler than WebSockets)
- Customize OpenAPI for better developer experience and documentation
- Mount legacy applications for gradual framework migration
- Isolate sub-applications with path prefixes and middleware
- Implement proper error boundaries around mounted apps

---

### Course Completion

You have completed **The Complete FastAPI Developer Handbook**. You now possess comprehensive knowledge of:

**Core Concepts**: Routing, dependency injection, Pydantic validation, async programming
**Security**: OAuth2, JWT, RBAC, password hashing, secure headers
**Databases**: SQLAlchemy async, MongoDB/Beanie, SQLModel, migrations
**Testing**: TestClient, async testing, dependency mocking, coverage
**Quality**: Ruff, Black, mypy, pre-commit hooks, CI/CD
**Real-Time**: WebSockets, SSE, background tasks
**Production**: Docker, Gunicorn, Nginx, cloud deployment, monitoring
**Advanced**: GraphQL, custom OpenAPI, sub-applications, error handling

**Next Steps:**
- Build production applications using these patterns
- Contribute to FastAPI ecosystem (issue reports, documentation)
- Explore specialized domains (microservices, event-driven architecture)
- Stay updated with FastAPI releases and Python async ecosystem evolution

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='23. graphql.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <span style='color:gray; font-size:1.05em;'>Next</span>
</div>
